# Deliverable 2
This is the notebook that has the second deliverable.

Table of Contents <br>
0. Import Modules
1. Data reading
2. Descriptive data analysis
3. Ways to enrich, filter, transform the data
4. Plan and Ideas for Methods

# 0. Import modules
Feel free to use the virtual environment (amonavis) included in the GitHub folder.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt 
import networkx as nx


# For semantic similarity
from urllib.parse import unquote
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

# Python functions in .py file to read data
import data_readers

# 1. Data reading
The following code reads the data and saves them in the appropriate variables.
<br><br>
**Wikispeedia**
This will hold our graph where Wikipedia articles are nodes and links/paths between them are edges.

<br><br>
**finished_paths**
The datafile includes the hashedIpAddress, timestamp, durationInSec, path, and rating of games that were completed.
We also add columns with the first article (soruce), last article (target), path length (#articles), and a readable date in Timestamp format.

<br><br>
**unfinished_paths**
This datafile is similar to finished_paths, but with games that weren't completed. It has the hashedIpAddress, timestamp, durationInSec, path, target, and type of failure (either timeout or restart).

<br><br>
**articles**
Dataframe with the name of all articles in the dataset.

<br><br>
**categories**
This shows the higher category classification of each article. For example, Áedán mac Gabráin is part of 'subject.History.British_History.British_History_1500_and_before_including_Roman_Britain'

In [ ]:
# The links and edges
wikispeedia = data_readers.read_wikispeedia_graph()

# The finished paths
finished_paths = data_readers.read_finished_paths()

# The unfinished paths
unfinished_paths = data_readers.read_unfinished_paths()

# List of all articles
articles = data_readers.read_articles()

# List of all articles and their categories
categories = data_readers.read_categories()


# We found out later that the data contained in the shortest path matrix given to us seems to be wrong
# Here we also add a quick dictionary that properly shows that this is wrong, and give an example
shortest_path_df = data_readers.read_shortest_path_df()
shortest_path_dict = dict(nx.all_pairs_shortest_path(wikispeedia))

# Searching for the string of a given article. It has to be formatted like the article name
# Which shouldn't be a problem, as we'll probably usually retrieve them internally
obi_wan_text = data_readers.plaintext_article_finder('Obi-Wan_Kenobi')

In [ ]:
print("Dataset has", len(wikispeedia.nodes), "nodes")
print("Dataset has", len(wikispeedia.edges), "edges")

These are less nodes than the reported number, it should be 4,604 nodes.

The 119,882 edges is correct though.

The difference is probably due to some nodes not being connected to the rest of the graph, as here we read in only the articles that are connected. The few nodes that we are losing do not matter for what we need.

In [ ]:
# Let's print a sample of each datafram to make sure they were read in correctly.
finished_paths.sample(5)

In [ ]:
unfinished_paths.sample(5)

In [ ]:
articles.sample(5)

In [ ]:
categories.sample(5)

In [ ]:
shortest_path_df.head()

# 2. Descriptive Data Analysis
Here, we show that we understand what’s in the data (formats, distributions, missing values, correlations, etc.).


In this section:
<br>2.1. How many articles there are, how many paths 
<br>2.2. Histograms of the links from each article (for example, how many articles have 20 links, etc)
<br>2.3. Average distance from one article to any other article
<br>2.4. Histogram of the number of games at each point in time
<br>2.5. Understanding unfinished games: Categories of targets in unfinished games


### 2.1. How many: articles, links, finished games, unfinished games?


In [ ]:
print("There are", len(wikispeedia.nodes), "articles in the dataset.")
print("There are", len(wikispeedia.edges), "links/paths.")
print("There are", finished_paths.shape[0], "finished games.")
print("There are", unfinished_paths.shape[0], "unfinished games.")

In [ ]:
unique_paths = finished_paths['path'].unique()
print('There are', len(unique_paths), 'unique finshed paths.')

### 2.2. Degree of a Node
The degree of a node is the number of edges/links it has. We plot a complementary cumulative distribution function (CCDF) of degree for each article/node.

In [ ]:
# Get the degrees of all nodes
degrees = dict(wikispeedia.degree())

# Calculate the CCDF
degree_values = sorted(set(degrees.values()), reverse=True)
ccdf = [sum(1 for degree in degrees.values() if degree >= d) for d in degree_values]

# Plot the CCDF
plt.plot(degree_values, ccdf, marker='o', linestyle='-', color='b')
plt.xscale('log')  # Use log scale for better visualization
plt.yscale('log')
plt.xlabel('Degree')
plt.ylabel('Complementary Cumulative Distribution Function (CCDF)')
plt.title('CCDF of Node Degrees')
plt.show()

What are the "hubs"? Which nodes have more than 500 links?

In [ ]:
print("Nodes with more than 1000 edges: ", [node for node in wikispeedia.nodes if wikispeedia.degree(node) >= 1000])
print("Nodes with more than 500 edges: ", [node for node in wikispeedia.nodes if wikispeedia.degree(node) >= 500])


Observe that biggest hubs are mainly countries. The 'United Kingdom', 'France', 'United States', and 'Europe' have over 1000 links. Since there are 4592 nodes, these nodes have link to almost 1/4 of the dataset!

How many nodes have more than 20 links? How many have only 1 link?

In [ ]:
# Count the number of nodes with degree 1
nodes_degree_1 = [node for node in wikispeedia.nodes if wikispeedia.degree(node) == 1]
print('Nodes with degree 1: ', nodes_degree_1)

# Get the percentages
total_nodes = len(wikispeedia.nodes)
percentage_degree_1 = (len(nodes_degree_1) / total_nodes)
print('% of nodes that have only 1 edge/link:', percentage_degree_1)

# Count number of nodes with degree <= 20
nodes_degree_20 = sum(1 for degree in degrees.values() if degree <= 20)
print('% of nodes that have 20 or less edges/links:', nodes_degree_20 / total_nodes)

### 2.3. Average Distance between Articles
On average, how many links/edges does it take to connect any random two articles?

In [ ]:
# Our graph is not strongly connected, meaning it's not possible to reach every node from every other node
# So we can't use the built in function nx.average_shortest_path_length

# This takes a long time to run (30 sec)
shortest_paths = list(nx.all_pairs_shortest_path_length(wikispeedia))
reachable_pairs = [(source, target, length) for source, paths in shortest_paths for target, length in paths.items() if length != float('inf')]
total_distances = sum(length for _, _, length in reachable_pairs)
average_distance = total_distances / len(reachable_pairs) if reachable_pairs else 0
print(f"Average distance between reachable nodes: {average_distance:.2f}")


### 2.4. Games per Time
Histogram of the number of games at each point in time.


In [ ]:
# Convert timestamps to datetime objects
timestamps = finished_paths['timestamp']
date_times = [datetime.utcfromtimestamp(ts) for ts in timestamps]

# Create a histogram of timestamps
plt.hist(date_times, bins=20, color='skyblue')
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.title('Distribution of Timestamps')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.show()

A lot of games were played in Q3 2009! 

In [ ]:
q3_2009_times = [dt for dt in date_times if dt.year == 2009 and dt.month in [7, 8, 9]]
percent_q3_2009_times = len(q3_2009_times) / len(date_times)
print(f"Percent of finished games completed in Q3 2009: {percent_q3_2009_times:.2f}")


### 2.5. Which categories of games are more likely to be unfinished? 

In [ ]:
print('There are', categories['article'].duplicated().sum(), 'articles with more than 1 category.')

print('Should we drop the duplicate categories, or doublecount them?')
print('This corresponds to', categories['article'].duplicated().sum() / len(wikispeedia.nodes), 'of the articles.')

# Let's drop them for now.
categories['article'] = categories['article'].drop_duplicates()
print('The new shape is: ',categories.shape)

# Why are there more articles here than nodes (# articles)?

In [ ]:
# Let's use string manipulation to extract the highest level category for each article.

sub_categories = categories['categories'].str[8:].str.split('.')
category_depth_1 = sub_categories.apply(lambda x: x[0])
categories['depth_1'] = category_depth_1
categories.head()

Let's find the category corresponding to each unfinished target.

In [ ]:
# Merging categories with unfinished paths.
unfinished_paths_with_categories = pd.merge(unfinished_paths, categories, left_on = 'target', right_on= 'article', how = 'left')

# Count the occurrences of each category
category_counts = unfinished_paths_with_categories['depth_1'].value_counts()

# Plotting the bar chart
plt.bar(category_counts.index, category_counts.values)

# Adding labels and title
plt.xlabel('Depth 1 Categories')
plt.ylabel('Frequency')
plt.title('Frequency of Categories of Target Articles in Unfinished Paths')

plt.xticks(rotation=90)  # Rotate x-axis labels for better readability
plt.tight_layout()  # Ensure labels fit within the plot area

Notice that targets in the science category make up the largest proportion of unfinished games. In Deliverable 3, we will investigate this more. We'll discover if this is because most of the articles are from the science category, or if science articles are actually harder to find in the game.

### 2.6 Issue with the shortest path
The shortest paths that were calculated are wrong. This is just a simple example to show that at least one of the points is wrong. We assume this is common enough elsewhere, as the effort needed to properly prove this is too large.

We double-checked the method for reading in the data, and there are no big changes to be done there. We also double-checked that the graph is a directed graph, which matters for this search. Additionally, we manually checked the edges, so we know that the following example is correct.

In [ ]:
# We have to put the strings into tuples to make this work. It's ugly, but this works
print("Shortest path from Actor to Japan in the given matrix is:", shortest_path_df[('Actor',)][('Japan',)])

print("Shortest path from Actor to Japan according to networkX is:", len(shortest_path_dict['Actor']['Japan'])-1)

print("The actual path is:", shortest_path_dict['Actor']['Japan'])

In [ ]:
# No need to print the entire text, just show that reading it in works
obi_wan_text[:100]

### 2.7. Study of Unique Paths
Here we study the unique source and target pairs. We will use the dataframes to compare the performance between humans and machines, as well as to know what paths to make machines complete.

**article_combinations**

This dataframe contains information on all the combination of source and target articles in the finished games (paths). It includes how many times it has been played, and the mean and std of the path length, duration of the game, and rating.

**unique_targets** and **unique_sources**

These dataframes include all the sources and targets that appears in the finished games

<br><br>
Note that we don't change to ASCII the name of the articles yet. We will do it at a later step if we need to.
<br><br>

In [ ]:
# How many each pair of articles has been visited
article_combinations_count = finished_paths.groupby(['first_article', 'last_article']).size().reset_index(name='count')

# The mean and std of the path length for each pair of articles
article_combinations_stats = finished_paths.groupby(['first_article', 'last_article'])['path_length'].agg(['mean', 'std']).reset_index()
article_combinations_stats['std'] = article_combinations_stats['std'].fillna(0)
article_combinations_stats.rename(columns={'mean': 'mean_length', 'std': 'std_length'}, inplace=True)

# The mean and std of the rating for each pair of articles. 
    # Note that mean and std may be nan if there are nan ratings. We purposely leave them as nan, as we don't want to fill them with 0s or 1s.
    # Depending on the application, we could change this in the future if neeeded.
rating_combinations_stats_rating = finished_paths.groupby(['first_article', 'last_article'])['rating'].agg(['mean', 'std']).reset_index()
#rating_combinations_stats_rating['std'] = rating_combinations_stats_rating['std'].fillna(0)
mask = rating_combinations_stats_rating['mean'].notnull()
rating_combinations_stats_rating.loc[mask, 'std'] = rating_combinations_stats_rating.loc[mask, 'std'].fillna(0)
rating_combinations_stats_rating.rename(columns={'mean': 'mean_rating', 'std': 'std_rating'}, inplace=True)

# The mean and std of the time for each pair of articles.
rating_combinations_stats_time = finished_paths.groupby(['first_article', 'last_article'])['durationInSec'].agg(['mean', 'std']).reset_index()
rating_combinations_stats_time['std'] = rating_combinations_stats_time['std'].fillna(0)
rating_combinations_stats_time.rename(columns={'mean': 'mean_durationInSec', 'std': 'std_durationInSec'}, inplace=True)

# Merging all the dataframes
article_combinations = pd.merge(article_combinations_count, article_combinations_stats, on=['first_article', 'last_article'])
article_combinations = pd.merge(article_combinations, rating_combinations_stats_rating, on=['first_article', 'last_article'])
article_combinations = pd.merge(article_combinations, rating_combinations_stats_time, on=['first_article', 'last_article'])

# The number of unique sources and targets
unique_sources = finished_paths['first_article'].value_counts().reset_index()
unique_targets = finished_paths['last_article'].value_counts().reset_index()

In [ ]:
article_combinations.sample(5)

In [ ]:
unique_sources.sample(5)

In [ ]:
unique_targets.sample(5)

# 3. Ways to enrich, filter, transform the data

## 3.1 Semantic Similarity

An important part of the project is to study how humans and machines move from article to article. Semantic similarity compares two strings based on a trained model and assigns a value according to how are they correlated (the higher, the more related). For example, 'king' and 'queen' will have a higher semantic similarity than say, 'king' and 'chemistry' (will prove this here).

### Remove the underscore and decode the url

First we define a function that corrects the strings to have a readable format. For example, '%C3%89douard_Manet' is transformed to 'Édouard Manet'.

We will create a function to decode a word, and we will be able to use it in series and dataframes using apply().

In [ ]:
def decode_word(word):
    word = word.replace('_', ' ')
    return unquote(word)

In [ ]:
articles['articles'].head(5)

In [ ]:
articles['articles'].apply(decode_word).head(5)

### Semantic Distance Model
We create a function that returns the semantic similarity between two words you provide.

In [ ]:
# We define the model outside the function (make sure to run this before using the function)
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# Function to get embeddings (just because we will use it in semantic similarity function)
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

# Semantic similarity function
def semantic_similarity(word1, word2):
    embedding1 = get_embedding(word1)
    embedding2 = get_embedding(word2)
    return cosine_similarity(embedding1.detach().numpy(), embedding2.detach().numpy())[0][0]

In [ ]:
semantic_similarity('king', 'queen')

In [ ]:
semantic_similarity('king', 'chemistry')

### Semantic Distance Matrix
Provided a series, it creates a df where indices and column names are the strings of the series, and fills the matrix with the semantic similarity between all words in the provided series.

In [ ]:
def semantic_similarity_matrix(titles):
    df = pd.DataFrame(index=titles, columns=titles)
    for i in range(len(titles)):
        for j in range(i+1, len(titles)):
            embedding1 = get_embedding(titles[i])
            embedding2 = get_embedding(titles[j])
            similarity = cosine_similarity(embedding1.detach().numpy(), embedding2.detach().numpy())[0][0]
            df.iloc[i, j] = similarity
            df.iloc[j, i] = similarity  # Copy value to lower triangle
            np.fill_diagonal(df.values, 1)
    return df

In [ ]:
semantic_similarity_matrix(pd.Series(['king', 'queen', 'chemistry', 'biology']))

# 4. Plan and Ideas for Methods


## 4.1 Basic AI using A* Algorigm

Most of the model requires for there to be an AI model we can compare it against.

We were indicated by the TA to not focus on this too much, as this is a data analysis course, not an ML course. Because of this, we took the implementation of A\* that was included in networkX and created two modified versions that now do the following:
- First version returns all of the explored nodes, not just the shortest path found
- Second version is forced to do a depth first search without being able to return

For our purposes, the explored nodes is the most interesting metric, as it describes what were the links "clicked".

We found a paper that implements a more complicated version, and we might be able to do something with graph neural networks, but for now this is good enough.

We additionally do a bit of work to show how the system works timewise, as well as how the comparison will work in the future.

The time comparison is not super useful though, as that will depend on hardware too much to be worth using easily.

In [ ]:
import machine_searchers
import time

def modded_get_embedding(text: str):
    temp_str = text.replace('_', ' ')
    temp_str = unquote(temp_str)
    inputs = tokenizer(temp_str, return_tensors="pt")
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

def distance_two_words(w1: str, w2: str):
    """Receives a string that was in the wikispeedia dataset, and transforms it as needed to work
    with the berd embeddings."""

    embedding1 = modded_get_embedding(w1)
    embedding2 = modded_get_embedding(w2)
    similarity = cosine_similarity(embedding1.detach().numpy(), embedding2.detach().numpy())[0][0]
    # Adding absolute, just in case it is needed
    # Similarity is actually 1 - abs(similarity) + 1,
    # As we want closer words to have a smaller distance
    # The last plus one is to indicate that there would be an extra cost to exploring, as if not the system often
    # thinks that there are nodes that have a distance of 0.5 or something like that
    similarity = 1 - abs(similarity) + 1
    # print("First word:", w1, ". Second word:", w2, ". GoodDistance:", similarity)
    return similarity

start_time = time.time()
lib_path_1, lib_explore_1 = machine_searchers.modded_astar_path(wikispeedia, 'Actor', 'Japan', heuristic=distance_two_words)
end_time = time.time()

# It's len - 1 because the target node is also included, and that node wasn't explored
print("Using the modded a star that returns explored nodes:")
print(" Found solution for Actor to Japan exploring the following number of nodes:", len(lib_explore_1)-1)
print(" Found it in:", end_time-start_time)

start_time = time.time()
lib_path_2, lib_explore_2 = machine_searchers.only_depth_first_astar_path(wikispeedia, 'Actor', 'Japan', heuristic=distance_two_words)
end_time = time.time()

# It's len - 1 because the target node is also included, and that node wasn't explored
print("Using depth first only A star that returns explored nodes:")
print(" Found solution for Actor to Japan exploring the following number of nodes:", len(lib_explore_1)-1)
print(" Found it in:", end_time-start_time)


Now we'll take the most commonly explored node pair path, run it through the two algorithms and see what is the result!

In [ ]:
# So it's finding the length between asteroid and viking
article_combinations.sort_values('count', ascending=False).head()

In [ ]:
start_time = time.time()
lib_path_1, lib_explore_1 = machine_searchers.modded_astar_path(wikispeedia, 'Asteroid', 'Viking', heuristic=distance_two_words)
end_time = time.time()

# It's len - 1 because the target node is also included, and that node wasn't explored
print("Using the modded a star that returns explored nodes:")
print(" Found solution for Asteroid to Viking exploring the following number of nodes:", len(lib_explore_1)-1)
print("Path length was:", len(lib_path_1)-1)
print(" Found it in:", end_time-start_time)

start_time = time.time()
lib_path_2, lib_explore_2 = machine_searchers.only_depth_first_astar_path(wikispeedia, 'Asteroid', 'Viking', heuristic=distance_two_words)
end_time = time.time()
print('')

# It's len - 1 because the target node is also included, and that node wasn't explored
print("Using depth first only A star that returns explored nodes:")
print(" Found solution for Asteroid to Viking exploring the following number of nodes:", len(lib_explore_2)-1)
print("Path length was:", len(lib_path_2)-1)
print(" Found it in:", end_time-start_time)

In [ ]:
lib_path_1

So, based on the previous example of the path, this works out well enough. The systems spends a lot of time exploring and going back, which might be a common issue. There is a huge disconnect between explored and actual path length, but that is common for A\*, so it's an expected caveat.

It is interesting to note that the optimal solution passed through Paris, which seems to fit the definition of being one of the hubs that are described in the paper. Maybe the hub strategy is actually useful in most cases!

The depth first method took a lot longer to run than planned. Based on this, it might be worth considering other alternatives.

But still, at least we've proven the model works, and can give results that we can compare against humans. Again, this is using a much simpler method, but this could be enhanced in the future.